In [ ]:
import json
import pandas as pd
import requests
from sqlalchemy import create_engine

def from_weather_api_2_mysql():
    
    city = 'Berlin'
    API_key = '4fe53ee5e34a7d900ed58bd74bbbb0b7'

    # check out the docs for more info on making an api call https://openweathermap.org/forecast5
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")

    response = requests.get(url)
    json = response.json()


    weather_dict = {'city': [],
                    'country': [],
                    'forecast_time': [],
                    'outlook': [],
                    'detailed_outlook': [],
                    'temperature': [],
                    'temperature_feels_like': [],
                    'clouds': [],
                    'rain': [],
                    'snow': [],
                    'wind_speed': [],
                    'wind_deg': [],
                    'humidity': [],
                    'pressure': []
                   }

    cities = ['Berlin', 'Frankfurt']
    for city in cities:
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        response = requests.get(url)
        json = response.json()

        for i in json['list']:
          weather_dict['city'].append(json['city']['name'])
          weather_dict['country'].append(json['city']['country'])
          weather_dict['forecast_time'].append(i['dt_txt'])
          weather_dict['outlook'].append(i['weather'][0]['main'])
          weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
          weather_dict['temperature'].append(i['main']['temp'])
          weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
          weather_dict['clouds'].append(i['clouds']['all'])
          try:
              weather_dict['rain'].append(i['rain']['3h'])
          except:
              weather_dict['rain'].append('0')
          try:
              weather_dict['snow'].append(i['snow']['3h'])
          except:
              weather_dict['snow'].append('0')
          weather_dict['wind_speed'].append(i['wind']['speed'])
          weather_dict['wind_deg'].append(i['wind']['deg'])
          weather_dict['humidity'].append(i['main']['humidity'])
          weather_dict['pressure'].append(i['main']['pressure'])

    df = pd.DataFrame(weather_dict)

    database = 'collected_data'

    username = 'root'

    password = 'root'

    host = '127.0.0.1'

    sqlEngine       = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}', pool_recycle=3600)

    dbConnection    = sqlEngine.connect()

    tableName = 'weather_city_info1'

    try:

        frame = df.to_sql(tableName, dbConnection, if_exists='append', index = False);

    except ValueError as vx:

        print(vx)

    except Exception as ex:   

        print(ex)

    else:

        print("The data was pushed sucessfully to Table %s."%tableName);   

    finally:

        dbConnection.close()
    
def lambda_handler(event, context):
    from_weather_api_2_mysql()
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

